# ***Trabajo Final del Curso introductorio de Python IEEE***
Tema a tratar: Análisis comparativo del mercado financiero
Se busca realizar un programa que realice un **estudio comparativo del desempeño financiero de distintas empresas**.

**Consignas:**



*   Realizar un grafico del valor de las acciones diarias de las compañias Amazon y Google utilizando dos tipos de lineas distintos. 

*   Colocar como una entrada del programa el intervalo de fechas que se desea gráficar.

*   Encontrar los puntos donde se cruzan los dos gráficos y marcarlos con puntos.

*   Almacenar en un archivo de excel las fechas donde ocurrieron las intersecciones.

*   Armar un programa donde se analize la información de ciertas empresas (entrada del usuario) y se las gráfique comparativamente.


*   Gráficar la derivada discreta de los valores de bolsa de cada empresa, esto es, la diferencia entre el valor actual y el anterior para cada punto en el tiempo.

*   Calcular cual es aquella que creció más y aquella que creció menos en el mes pasado (Marzo), en el mes anterior (Febrero) y los últimos 12 meses, almacenar la información en un archivo excel.

**Información del alumno:**

Nombre: Agustín Ignacio Vergniaud

Legajo: 60341

Es factible remarcar que busqué resolver las consignas de una manera genérica. Los puntos solicitados para **Google** y **Amazon** funcionan para estas empresas y para cualquier otra de **NASDAQ**. El analisis de crecimiento también se realizó de manera generica, dejando al usuario decidir sobre que intervalo de tiempo quiere analizar, y no solamente sobre Marzo, Febrero o asimismo 12 meses atrás.

In [1]:
pip install pandas

In [2]:
pip install yfinance

     |████████████████████████████████| 5.5MB 6.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=e6b22483510044f307d92d4012d6716e70d6512ef7d6d99516d0d754d16d01a8
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


En este bloque le daré forma a Empresas, que en su interior contendrá los nombres de las empresas a analizar y sus códigos de NASDAQ.
A su vez, utilizaré mi referencia a todas las companias existentes en  NASDAQ con el archivo https://pkgstore.datahub.io/core/nasdaq-listings/nasdaq-listed_csv/data/7665719fb51081ba0bd834fde71ce822/nasdaq-listed_csv.csv

---



In [ ]:
! wget https://pkgstore.datahub.io/core/nasdaq-listings/nasdaq-listed_csv/data/7665719fb51081ba0bd834fde71ce822/nasdaq-listed_csv.csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cbook as cbook
import matplotlib.dates as mdates
#Funcion general:
#	Que dados los nombres de varias empresas, las compara en un gráfico.
	

empresas = {
	"nombres" : [],
	"simbolo" : [],
}
listings = pd.read_csv("nasdaq-listed_csv.csv")


Aquí establezco la función **pushdata**, donde cargaré el diccionario empresas con la información pertinente. Es clave remarcar que "codigo" será una lista de empresas, ya que yo querré graficar y comparar una cantidad indefinida de empresas entre sí.

In [ ]:
def pushData(codigo):
	for k in listings.index:
		for i in range(len(codigo)): #Ya que nombre será una lista
			if (codigo[i] == listings["Symbol"][k]):
				empresas["nombres"].append(listings["Company Name"][k])
				empresas["simbolo"].append(codigo[i])

Señalo función con la que voy a graficar y ordenar las fechas, de modo de que no se superpongan en el eje x al graficar.

In [ ]:
import pylab
import matplotlib
import matplotlib.dates as dates
import time
import datetime
#Función para graficar y evadir superposiciones en el eje x
def plotDates(x, y,subplots,codigoEmpresa):
	fig, ax = subplots

	df = pd.DataFrame(
        data=y, 
        index=pd.to_datetime(x),
        columns=['A']
	)

	formatter = dates.DateFormatter('%Y-%m-%d') 
	ax.xaxis.set_major_formatter(formatter)
	plt.plot(df.index, df['A'],label=codigoEmpresa)
	fig.autofmt_xdate()
	plt.title("Comparación de valor de acciones",size=20)
	plt.ylabel("Valor de Acciones (USD)",size=20)
	plt.legend()

Con el motivo de diferenciar el **gráfico de la comparación** entre el crecimiento de las empresas creo otra función.

In [ ]:
def plotDerivadas(x, y,subplots,codigoEmpresa):
	fig, ax = subplots

	df = pd.DataFrame(
        data=y, 
        index=pd.to_datetime(x),
        columns=['A']
	)

	formatter = dates.DateFormatter('%Y-%m-%d') 
	ax.xaxis.set_major_formatter(formatter)
	plt.plot(df.index, df['A'],label=codigoEmpresa)
	fig.autofmt_xdate()
	plt.title("Comparación de crecimiento temporal del valor de acciones",size=20)
	plt.ylabel("Valor de Acciones (USD)",size=20)
	plt.legend()

Aquí agrego otra función para graficar los puntos de intersección, en caso de que el usuario añada **dos** empresas para comparar entre sí. De esta manera puedo resolver el ejercicio entre **GOOGLE** y **AMAZON** solicitado de manera general. El programa graficará las **dos** empresas que el usuario solicite y encontrará sus puntos de intersección.

In [ ]:
def plotDots(x, y,subplots):
	fig, ax = subplots

	df = pd.DataFrame(
        data=y, 
        index=pd.to_datetime(x),
        columns=['A']
	)

	formatter = dates.DateFormatter('%Y-%m-%d') 
	ax.xaxis.set_major_formatter(formatter)
	plt.plot(df.index, df['A'],'o',label="Intersección")

Descarga de datos de **YAHOO FINANCE** y prevención errores mediante el ingreso de datos erroneos.

In [ ]:
import yfinance as yf
import sys
#Conversión a csv de los datos de Yahoo Finance
#Evitar el ingreso de datos incorrectos
def guardarDatos(codigoEmpresa,desde,hasta):
	pushData(codigoEmpresa)
	try:
		datetime.datetime.strptime(desde, '%Y-%m-%d')
	except:
		sys.exit("Ingrese la fecha inicial en el formato YY-MM-DD")
	try:
		datetime.datetime.strptime(hasta, '%Y-%m-%d')
	except:
		sys.exit("Ingrese la fecha final en el formato YY-MM-DD")
	for k in range(len(empresas["simbolo"])):
		if codigoEmpresa[k] not in empresas["simbolo"]:
			return("No se encontro la empresa")
		else:
			infoempresa = yf.Ticker(str(empresas["simbolo"][k]))
			infoempresa.history(period="5y")
			data_df = yf.download(str(empresas["simbolo"][k]), start=desde, end=hasta)
			data_df.to_csv(str(empresas["simbolo"][k])+".csv")

Aquí introduzco una función que utilizaré luego con el fin de analizar el **crecimiento** de las acciones de las empresas solicitadas en función del tiempo.

In [ ]:
def derivada(y,derivy):
  for i in range(len(y)):
    derivy.append(y[i]-y[i-1])

Ingreso de datos en los vectores que definirán la forma de mi gráfico en forma de abscisa y ordenada para la cantidad de empresas que solicite analizar.

In [ ]:
#Datos de abscisa y ordenada para las empresas requeridas
def lecturaDatos(codigoEmpresa,desde,hasta):
	guardarDatos(codigoEmpresa,desde,hasta)
	compania={
	}
	x={
	}
	y={
	}
	inter={
			"Date":[],
			"Open":[],
	}
	cx=[]
	cy=[]
	derivx={
	}
	derivy={
	}
	for k in empresas["simbolo"]:
		datos = pd.read_csv(str(k)+".csv")
		compania[k] = datos.to_dict('list')	
	# Creo keys y valores vacíos en mis diccionarios para utilizarlos luego
		x[k]=[]
		y[k]=[]
		derivx[k]=[]
		derivy[k]=[]
		if desde not in compania[k]["Date"]:
			sys.exit("La fecha "+ desde +" que usted ingreso no se encuentra en los archivos de NASDAQ para alguna de las empresas seleccionadas")
		for i in range(len(compania[k]["Date"])):
			x[k].append(compania[k]["Date"][i])
			y[k].append(compania[k]["Open"][i])
	subplots = plt.subplots(figsize=(10, 6))
	for k in empresas["simbolo"]:
		plotDates(x[k],y[k],subplots,k)
	if len(codigoEmpresa)==2:
		for i in range(len(x[empresas["simbolo"][0]])): #Ya que son vectores paralelos los x
			if (y[empresas["simbolo"][0]][i] == y[empresas["simbolo"][1]][i]) or (y[empresas["simbolo"][0]][i] > y[empresas["simbolo"][1]][i] and y[empresas["simbolo"][0]][i-1] < y[empresas["simbolo"][1]][i-1]) or (y[empresas["simbolo"][0]][i] < y[empresas["simbolo"][1]][i] and y[empresas["simbolo"][0]][i-1] > y[empresas["simbolo"][1]][i-1]):				
				cx.append(x[empresas["simbolo"][0]][i])
				cy.append(y[empresas["simbolo"][0]][i])
				#Para luego introducirlos en excel
				inter["Date"].append(x[empresas["simbolo"][0]][i])
				inter["Open"].append(y[empresas["simbolo"][0]][i])
		plotDots(cx,cy,subplots)
		#De esta manera paso la informacion sobre puntos de interseccion al archivo excel
		puntos_interseccion = pd.DataFrame.from_dict(inter, orient="columns")		
		puntos_interseccion.to_excel("puntos_interseccion.xlsx")
	#Para satisfacer consigna de derivada
	grafiderivada=plt.subplots(figsize=(9, 6))
	for k in empresas["simbolo"]:
		derivada(y[k],derivy[k])
		plotDerivadas(x[k],derivy[k],grafiderivada,k)

In [ ]:
def sumaLista(listaNumeros):
    laSuma = 0.
    for i in listaNumeros:
        laSuma = laSuma + i
    return laSuma

Aquí haré un método genérico para evaluar el crecimiento de una empresa en el período de tiempo que uno desee con el proposito de salisfacer la consigna solicitada. Evalúo la **suma de las derivadas** de cada función en función del tiempo.

In [ ]:
import pandas as pd
def crecimientoTemporal(codigoEmpresa,desde,hasta):
  guardarDatos(codigoEmpresa,desde,hasta)
  datosaComparar={
	}
  derivy={
	}
  y={
  }
  sumaderivada={
  }
  listasimbolos=[]
  listadatos=[]
  mini=0.
  maxi=0.
  punteroNombremax=-1
  punteroNombremin=-1
  for k in empresas["simbolo"]:
    datos = pd.read_csv(str(k)+".csv")
    datosaComparar[k] = datos.to_dict('list')	
	# Creo keys y valores vacíos en mis diccionarios para utilizarlos luego
    y[k]=[]
    sumaderivada[k]=[]
    derivy[k]=[]
    if desde not in datosaComparar[k]["Date"]:
      sys.exit("La fecha "+ desde +" que usted ingreso no se encuentra en los archivos de NASDAQ para alguna de las empresas seleccionadas")
    for i in range(len(datosaComparar[k]["Date"])):
      y[k].append(datosaComparar[k]["Open"][i])
    derivada(y[k],derivy[k])  
    for i in range (len(derivy[k])):
      sumaderivada[k].append(derivy[k][i])
    suma=sumaLista(sumaderivada[k])
    listadatos.append(suma)
    listasimbolos.append(k)
  maxi=max(listadatos)
  mini=min(listadatos)
  #listadatos y listanombres son vectores paralelos
  for i in range(len(listadatos)):
    if maxi==listadatos[i]:
      punteroNombremax=i
  for i in range(len(listadatos)):
    if min==listadatos[i]:
      punteroNombremin=i
  print("El valor máximo de crecimiento en el periodo de tiempo seleccionado es "+ str(maxi) + " y es de la empresa " + listasimbolos[punteroNombremax])
  print("El valor mínimo de crecimiento en el periodo de tiempo seleccionado es "+ str(mini) + " y es de la empresa " + listasimbolos[punteroNombremin])


In [ ]:
#Evito acumulación de datos de Google Collab
empresas["nombres"]=[]
empresas["simbolo"]=[]
empresa=[]
respuesta = "si"
while respuesta=="si":
  try:
    codigo=str(input("Ingrese simbolo NASDAQ de empresa a evaluar "))
    empresa.append(codigo)
    respuesta=str(input("Quiere seguir ingresando empresas? ")).lower()
  except:
    sys.exit("Por favor ingrese letras")
desde=input("Ingrese fecha inicial para comparar los valores de las acciones de las empresas ")
hasta=input("Ingrese fecha final para comparar los valores de las acciones de las empresas ")
lecturaDatos(empresa,desde,hasta)
#Evito acumulación de datos de Google Collab
empresas["nombres"]=[]
empresas["simbolo"]=[]
desdecrec=input("Ingrese fecha inicial para comparar crecimientos de las acciones de las empresas ")
hastacrec=input("Ingrese fecha final para comparar crecimientos de las acciones de las empresas ")
crecimientoTemporal(empresa,desdecrec,hastacrec)